## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Saint-Augustin,CA,51.23,-58.65,41.86,62,19,7.25,few clouds
1,1,Georgetown,MY,5.41,100.34,77.00,94,40,3.36,moderate rain
2,2,Arraial Do Cabo,BR,-22.97,-42.03,71.60,88,0,9.17,clear sky
3,3,Hermanus,ZA,-34.42,19.23,53.83,80,14,4.92,few clouds
4,4,Bredasdorp,ZA,-34.53,20.04,44.60,93,19,1.12,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
1,1,Georgetown,MY,5.41,100.34,77.00,94,40,3.36,moderate rain
8,8,Taoudenni,ML,22.68,-3.98,89.64,18,15,12.80,few clouds
12,12,Bathsheba,BB,13.22,-59.52,77.00,83,20,5.82,few clouds
15,15,Victoria,HK,22.29,114.16,87.01,81,97,1.01,light rain
16,16,Vila Velha,BR,-20.33,-40.29,75.99,78,0,11.41,clear sky
19,19,Montecristo,CO,8.30,-74.47,75.15,91,84,1.23,broken clouds
24,24,Hithadhoo,MV,-0.60,73.08,82.56,78,69,9.24,broken clouds
27,27,Bengkulu,ID,-3.80,102.27,83.84,66,40,5.53,scattered clouds
30,30,San Miguel,SV,13.48,-88.18,75.29,92,99,1.97,overcast clouds
33,33,Muli,MV,2.92,73.57,82.53,77,73,9.37,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                203
City                   203
Country                203
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current description    203
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy()
clean_df.replace('', np.nan, inplace=True)
clean_df.dropna(inplace=True)


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
1,Georgetown,MY,77.00,moderate rain,5.41,100.34,NaN
8,Taoudenni,ML,89.64,few clouds,22.68,-3.98,NaN
12,Bathsheba,BB,77.00,few clouds,13.22,-59.52,NaN
15,Victoria,HK,87.01,light rain,22.29,114.16,NaN
16,Vila Velha,BR,75.99,clear sky,-20.33,-40.29,NaN
19,Montecristo,CO,75.15,broken clouds,8.30,-74.47,NaN
24,Hithadhoo,MV,82.56,broken clouds,-0.60,73.08,NaN
27,Bengkulu,ID,83.84,scattered clouds,-3.80,102.27,NaN
30,San Miguel,SV,75.29,overcast clouds,13.48,-88.18,NaN
33,Muli,MV,82.53,broken clouds,2.92,73.57,NaN


In [8]:
hotel_df.count()

City                   203
Country                203
Max Temp               203
Current description    203
Lat                    203
Lng                    203
Hotel Name               0
dtype: int64

In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [33]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current description} and {Max Temp} °F</dd>

</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))